# ĐỒ ÁN CUỐI KỲ - NHẬP MÔN KHOA HỌC DỮ LIỆU

## Đề tài: Ước lượng giá laptop

### Sinh viên:
        1. MSSV: 18120058        Tên: Phạm Công Minh
        2. MSSV: 18120090        Tên: Phạm Nguyên Minh Thy

---

## Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
#from sklearn.neural_network import MLPRegressor
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

In [2]:
import sklearn
sklearn.__version__

'0.23.1'

## Đọc dữ liệu

In [3]:
# Do data lấy theo tứ tự producer nên cần phải shuffle
from sklearn.utils import shuffle
data_df = pd.read_csv('data.csv')
data_df = shuffle(data_df, random_state=0)
data_df.head()

,producer,processor prod,processor model,cores,core base speed (GHz),core boost speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),...,gpu boost speed (GHz),screen type,screen size (inch),screen horizontal resolution,screen vertical resolution,sRGB (%),weight (kg),os,battery capacity (WHr),price(USD)
299,dell,intel,i5,4,1.6,4.2,ddr3,8,0,256,...,1100,led ips,13.3,1920,1080,90,1.24,windows home 10.00,45.0,723.0
500,hp,intel,i7,4,1.8,4.9,ddr4,8,0,512,...,1150,led ips,15.6,1920,1080,60,2.05,windows home 10.00,53.2,885.5
303,dell,intel,i5,4,2.4,4.2,ddr4,8,0,256,...,1300,led ips,14.5,2560,1600,100,1.26,windows home 10.00,52.0,730.0
40,acer,intel,i7,4,1.8,4.6,ddr3,8,0,256,...,1468,led ips,14.0,1920,1080,100,1.35,windows home 10.00,48.0,949.0
495,hp,intel,i7,4,1.8,4.6,ddr4,8,0,256,...,1150,led tn,17.3,1600,900,80,2.45,windows home 10.00,41.0,830.0


## Ý nghĩa các cột

    1. producer                         tên nhà sản xuất laptop
    2. processor prod                   nhà sản xuất CPU
    3. processor model                  dòng CPU
    4. cores                            số nhân của CPU
    5. core base speed (GHz)            tốc độ xung nhịp cơ bản của CPU
    6. core boost speed (GHz)           tốc độ xung nhịp tối đa của CPU
    7. ram type                         loại RAM
    8. ram cap (GB)                     dung lượng RAM
    9. ssd (GB)                         dung lượng SSD
    10. hdd (GB)                        dung lượng HDD
    11. gpu prod                        nhà sản xuất card màn hình
    12. gpu size (MB)                   dung lượng card màn hình
    13. gpu base speed (GHz)            tốc độ xung nhịp cơ bản của GPU
    14. gpu boost speed (GHz)           tốc độ xung nhịp tối đa của GPU
    15. screen type                     loại màn hình
    16. screen size (inch)              kích thước màn hình
    17. creen horizontal resolution     độ phân giải màn hình theo chiều ngang
    18. screen vertical resolution      độ phân giải màn hình theo chiều dọc
    19. sRGB (%)                        độ phủ màu của màn hình dựa trên tiêu chuẩn sRGB
    20. weight (kg)                     trọng lượng máy
    21. os                              hệ điều hành
    22. battery capacity (Whr)          dung lượng pin
    23. price(USD)                      giá

---

# Khám phá dữ liệu

In [4]:
# Dữ liệu có bao nhiêu dòng, bao nhiêu cột?
data_df.shape

(800, 23)

In [5]:
# Dữ liệu có dòng bị lặp không?
data_df.index.duplicated().sum()

0

In [6]:
# Cột output hiện có kiểu dữ liệu gì?
data_df["price(USD)"].dtype

dtype('float64')

In [7]:
# Cột output có giá trị thiếu không?
data_df["price(USD)"].isna().sum()

0

## Câu hỏi cần trả lời:

**Câu hỏi có dạng:** *Tính giá tiền laptop (`price(USD)`) dựa trên các thuộc tính của laptop*

**Lợi ích khi trả lời câu hỏi:** Dự đoán giá laptop dựa trên các thuộc tính giúp cho việc lựa chọn mẫu khi có dự định mua laptop, có 2 hướng đưa ra quyết định:
- Chọn trước các mẫu phù hợp túi tiền rồi lấy cấu hình tính ra giá dự đoán và so sánh với giá niêm yết của cửa hàng => chọn mẫu laptop có tỷ lệ giá thực tế/giá dự đoán thấp nhất.
- Đưa ra những cấu hình mong muốn để dự đoán giá của máy tính rồi từ khoảng giá tiền đó tìm những mẫu máy tính có những thuộc tính gần với mong muốn.

** Đây chỉ là một trong nhiều yếu tố để quyết định việc chọn mẫu laptop phù hợp.*

**Nguồn cảm hứng của câu hỏi:**
- Việc chọn cách định giá một sản phẩm nào đó lấy cảm hứng từ các bài làm khác đã thấy qua (qua thầy giới thiệu việc tính giá nhà, các bài tập Machine Learning trên mạng,...).
- Việc chọn laptop để dự đoán giá lấy cảm hứng từ chính bản thân khi đã từng phân vân trong việc chọn mẫu laptop để mua và chưa có những cơ sở đủ mạnh để đưa ra quyết định.

---

# Tiền xử lý

## Tách các tập

In [8]:
# Tách X và y
y_sr = data_df["price(USD)"]
X_df = data_df.drop("price(USD)", axis=1)

In [9]:
# Tách từ dữ liệu ra tập test theo tỉ lệ 50%:50%
X_df, test_X_df, y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.5, random_state=0)
# Từ 70% dữ liệu còn lại tách tập train và tập validation theo tỉ lệ 70%:30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

---

## Khám phá dữ liệu (tập huấn luyện)

In [10]:
# Xem kiểu dữ liệu của các cột
train_X_df.dtypes

producer                         object
processor prod                   object
processor model                  object
cores                             int64
core base speed (GHz)           float64
core boost speed (GHz)          float64
ram type                         object
ram cap (GB)                      int64
ssd (GB)                          int64
hdd (GB)                          int64
gpu prod                         object
gpu size (MB)                     int64
gpu base speed (GHz)              int64
gpu boost speed (GHz)             int64
screen type                      object
screen size (inch)              float64
screen horizontal resolution      int64
screen vertical resolution        int64
sRGB (%)                          int64
weight (kg)                     float64
os                               object
battery capacity (WHr)          float64
dtype: object

Các cột đều có kiểu dữ liệu phù hợp.

##### Xét sự phân bố giá trị của các thuộc tính dạng số

In [11]:
num_cols = ['cores',
            'core base speed (GHz)', 
            'core boost speed (GHz)',
            'ram cap (GB)',
            'ssd (GB)', 'hdd (GB)', 
            'gpu size (MB)', 
            'gpu base speed (GHz)',
            'gpu boost speed (GHz)',
            'screen size (inch)',
            'screen horizontal resolution',
            'screen vertical resolution',
            'sRGB (%)',
            'weight (kg)',
            'battery capacity (WHr)']

df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

,cores,core base speed (GHz),core boost speed (GHz),ram cap (GB),ssd (GB),hdd (GB),gpu size (MB),gpu base speed (GHz),gpu boost speed (GHz),screen size (inch),screen horizontal resolution,screen vertical resolution,sRGB (%),weight (kg),battery capacity (WHr)
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
min,2.0,1.0,3.0,4.0,0.0,0.0,128.0,200.0,500.0,12.3,1366.0,768.0,0.0,0.85,30.0
lower_quartile,4.0,1.6,3.9,8.0,0.0,256.0,1700.0,300.0,1100.0,14.0,1920.0,1080.0,60.0,1.40,45.0
median,4.0,2.1,4.2,8.0,0.0,256.0,2048.0,400.0,1170.0,15.6,1920.0,1080.0,90.0,1.80,52.5
upper_quartile,6.0,2.6,4.5,16.0,0.0,512.0,6144.0,1100.0,1442.0,15.6,1920.0,1080.0,100.0,2.10,66.0
max,8.0,3.0,5.1,128.0,2000.0,6144.0,16384.0,1531.0,1725.0,17.3,3840.0,2400.0,100.0,4.70,100.0


##### Xét sự phân bố giá trị của các thuộc tính không phải dạng số

In [12]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

,processor prod,processor model,ram type,screen type,producer,gpu prod,os
missing_ratio,0,0,0,0,0,0,0
num_values,2,6,2,4,6,3,6
value_ratios,"{'intel': 86.4, 'amd': 13.6}","{'i7': 42.5, 'i5': 37.5, 'ryzen5': 9.6, 'i3': 6.4, 'ryzen3': 2.1, 'ryzen7': 1.8}","{'ddr4': 96.4, 'ddr3': 3.6}","{'led ips': 88.2, 'led tn': 10.0, 'oled': 1.4, 'led tn wva': 0.4}","{'lenovo': 28.2, 'dell': 21.1, 'asus': 20.7, 'hp': 19.6, 'acer': 8.9, 'apple': 1.4}","{'intel': 53.2, 'nvidia': 33.9, 'amd': 12.9}","{'windows home 10.00': 62.1, 'windows pro 10.00': 32.5, 'linux ubuntu 0.00': 2.5, 'macos 11.00': 1.4, 'no os 0.00': 1.1, 'chrome os 0.00': 0.4}"


##### Xét sự phân bố giá trị của Output

In [13]:
train_y_sr.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

missing_ratio        0.0
min                319.0
lower_quartile     750.5
median            1000.0
upper_quartile    1399.6
max               4300.0
Name: price(USD), dtype: float64

Kết luận: Có vẻ là không có gì bất thường.

## Tiền xử lý (tập huấn luyện)

Các bước tiền xử lý đầu tiền (tách, xóa cột):
- Tách `processor model` ra thành 2 cột: 1 cột gồm các processor của intel, 1 cột gồm các processor của amd; do cách tách này nên cột `processor prod` không còn cần thiết nữa. Vì muốn chuyển cột `processor model` thành thuộc tính dạng số có **thứ tự** mà các giá trị gồm các dòng chip của 2 nhà sản xuất khác nhau nên việc sắp thứ tự chung với nhau là điều không hợp lý => tách ra thành 2 cột của từng nhà sản xuất rồi mới đánh thứ tự với từng cột 
- 2 cột `producer` và `os` có nhiều giá trị khác nhau nên sẽ chọn các giá trị xuất hiện nhiều nhất theo `num_top_producer` và `num_top_os` các giá trị khác được thay bằng giá trị "others".
- Do đã chọn thuộc tính từ trước ở bước thu thập dữ liệu nên không có thuộc tính nào không cần thiết

In [14]:
train_X_df.head()

,producer,processor prod,processor model,cores,core base speed (GHz),core boost speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),...,gpu base speed (GHz),gpu boost speed (GHz),screen type,screen size (inch),screen horizontal resolution,screen vertical resolution,sRGB (%),weight (kg),os,battery capacity (WHr)
495,hp,intel,i7,4,1.8,4.6,ddr4,8,0,256,...,300,1150,led tn,17.3,1600,900,80,2.45,windows home 10.00,41.0
73,acer,intel,i7,6,2.6,4.5,ddr4,32,0,1024,...,1380,1590,led ips,17.3,3840,2160,100,4.50,windows pro 10.00,72.0
39,acer,intel,i5,4,1.1,3.7,ddr4,16,0,512,...,300,1050,led ips,13.5,2256,1504,90,1.20,windows home 10.00,56.0
541,hp,intel,i7,4,1.8,4.9,ddr4,16,0,512,...,1519,1582,led ips,15.6,3840,2160,90,2.18,windows home 10.00,84.0
600,lenovo,intel,i5,4,1.0,3.6,ddr4,8,0,256,...,300,1050,led ips,14.0,1920,1080,60,1.50,windows home 10.00,57.0


In [15]:
# Hàm định nghĩa transformer tách giá trị của thuộc tính processor_model thành 2 cột: 
# - 1 cột gồm các processor của intel
# - 1 cột gồm các processor của amd
# Và chuyển về dạng số bằng phương pháp ranking
# Đồng thời xóa 2 cột "processor prod" và "processor model"
def split_processor_model(X):
    intel_processor_rank = {'i3': 1,
                            'i5': 2,
                            'i7': 3,
                            'ryzen3': 0,
                            'ryzen5': 0,
                            'ryzen7': 0}
    intel_processor_model = X['processor model'].replace(intel_processor_rank)


    amd_processor_rank = {'i3': 0,
                          'i5': 0,
                          'i7': 0,
                          'ryzen3': 1,
                          'ryzen5': 2,
                          'ryzen7': 3}
    amd_processor_model = X['processor model'].replace(amd_processor_rank)
    
    return X.assign(intel_processor_model=intel_processor_model, amd_processor_model=amd_processor_model).drop(["processor prod", "processor model"], axis=1)

In [16]:
# TEST
split_processor_model(train_X_df).head()

,producer,cores,core base speed (GHz),core boost speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),gpu prod,gpu size (MB),...,screen type,screen size (inch),screen horizontal resolution,screen vertical resolution,sRGB (%),weight (kg),os,battery capacity (WHr),intel_processor_model,amd_processor_model
495,hp,4,1.8,4.6,ddr4,8,0,256,intel,1700,...,led tn,17.3,1600,900,80,2.45,windows home 10.00,41.0,3,0
73,acer,6,2.6,4.5,ddr4,32,0,1024,nvidia,8192,...,led ips,17.3,3840,2160,100,4.50,windows pro 10.00,72.0,3,0
39,acer,4,1.1,3.7,ddr4,16,0,512,intel,8192,...,led ips,13.5,2256,1504,90,1.20,windows home 10.00,56.0,2,0
541,hp,4,1.8,4.9,ddr4,16,0,512,nvidia,2048,...,led ips,15.6,3840,2160,90,2.18,windows home 10.00,84.0,3,0
600,lenovo,4,1.0,3.6,ddr4,8,0,256,intel,8192,...,led ips,14.0,1920,1080,60,1.50,windows home 10.00,57.0,2,0


In [17]:
# Hàm chọn các giá trị xuất hiện nhiều nhất theo "num_top_producer" đối với cột "producer" và "num_top_os" đối với cột "os"
# và các giá trị khác được thay bằng giá trị "others"
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_producers=1, num_top_os=1):
        self.num_top_producers = num_top_producers
        self.num_top_os = num_top_os
    def fit(self, X_df, y=None):
        producer_col = X_df.producer
        self.producer_counts_ = producer_col.value_counts()
        producers = list(self.producer_counts_.index)
        self.top_producers_ = producers[:max(1, min(self.num_top_producers, len(producers)))]
        
        os_col = X_df.os
        self.os_counts_ = os_col.value_counts()
        os = list(self.os_counts_.index)
        self.top_os_ = os[:max(1, min(self.num_top_os, len(os)))]
        
        return self
    def transform(self, X_df, y=None):
        X = X_df.copy()
        X.loc[:, "producer"].replace(list(set(X.producer.unique())-set(self.top_producers_)), 'others', inplace=True)
        X.loc[:, "os"].replace(list(set(X.os.unique())-set(self.top_os_)), 'others', inplace=True)
        return X

In [18]:
# TEST
col_adderdropper = ColAdderDropper(num_top_producers=4, num_top_os=4)
col_adderdropper.fit(split_processor_model(train_X_df))
print(col_adderdropper.producer_counts_)
print()
print(col_adderdropper.top_producers_)
print()
print(col_adderdropper.os_counts_)
print()
print(col_adderdropper.top_os_)

lenovo    79
dell      59
asus      58
hp        55
acer      25
apple      4
Name: producer, dtype: int64

['lenovo', 'dell', 'asus', 'hp']

windows home 10.00    174
windows pro 10.00      91
linux ubuntu 0.00       7
macos  11.00            4
no os  0.00             3
chrome os  0.00         1
Name: os, dtype: int64

['windows home 10.00', 'windows pro 10.00', 'linux ubuntu 0.00', 'macos  11.00']


In [19]:
fewer_cols_train_X_df = col_adderdropper.transform(split_processor_model(train_X_df))

In [20]:
# Kiểm tra lại cột "os" đã xử lý đúng chưa
fewer_cols_train_X_df.os.unique()

array(['windows home 10.00', 'windows pro 10.00', 'linux ubuntu 0.00',
       'macos  11.00', 'others'], dtype=object)

In [21]:
# Kiểm tra lại cột "producer" đã xử lý đúng chưa
fewer_cols_train_X_df.producer.unique()

array(['hp', 'others', 'lenovo', 'dell', 'asus'], dtype=object)

In [22]:
fewer_cols_train_X_df.head()

,producer,cores,core base speed (GHz),core boost speed (GHz),ram type,ram cap (GB),ssd (GB),hdd (GB),gpu prod,gpu size (MB),...,screen type,screen size (inch),screen horizontal resolution,screen vertical resolution,sRGB (%),weight (kg),os,battery capacity (WHr),intel_processor_model,amd_processor_model
495,hp,4,1.8,4.6,ddr4,8,0,256,intel,1700,...,led tn,17.3,1600,900,80,2.45,windows home 10.00,41.0,3,0
73,others,6,2.6,4.5,ddr4,32,0,1024,nvidia,8192,...,led ips,17.3,3840,2160,100,4.50,windows pro 10.00,72.0,3,0
39,others,4,1.1,3.7,ddr4,16,0,512,intel,8192,...,led ips,13.5,2256,1504,90,1.20,windows home 10.00,56.0,2,0
541,hp,4,1.8,4.9,ddr4,16,0,512,nvidia,2048,...,led ips,15.6,3840,2160,90,2.18,windows home 10.00,84.0,3,0
600,lenovo,4,1.0,3.6,ddr4,8,0,256,intel,8192,...,led ips,14.0,1920,1080,60,1.50,windows home 10.00,57.0,2,0


   

Các bước tiền xử lý tiếp theo: (như Bài tập 3)
- Với các cột dạng số (`nume_cols`), ta sẽ điền giá trị thiếu bằng giá trị mean của cột (dùng `SimpleImputer`). Với *tất cả* các cột dạng số trong tập huấn luyện, ta đều cần tính mean, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới. 
- Với các cột không phải dạng số và không có thứ tự (`unorder_cate_cols`):
    - Ta sẽ điền giá trị thiếu bằng giá trị mode (giá trị xuất hiện nhiều nhất) của cột (dùng `SimpleImputer`). Với *tất cả* các cột không có dạng số và không có thứ tự, ta đều cần tính mode, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới.
    - Sau đó, ta sẽ chuyển sang dạng số bằng phương pháp mã hóa one-hot (dùng `OneHotEncoder`).
- Với cột không phải dạng số và có thứ tự (`order_cate_cols`):
    - Ta sẽ điền giá trị thiếu bằng giá trị mode (giá trị xuất hiện nhiều nhất) của cột.
    - Cột này đã được chuyển sang dạng số rồi nên ta không cần chuyển nữa.
- Cuối cùng, khi tất cả các cột đã được điền giá trị thiếu và đã có dạng số, ta sẽ tiến hành chuẩn hóa bằng cách trừ đi mean và chia cho độ lệch chuẩn của cột để giúp cho các thuật toán cực tiểu hóa như Gradient Descent, LBFGS, ... hội tụ nhanh hơn (dùng `StandardScaler`).

In [23]:
nume_cols = ['cores',
            'core base speed (GHz)', 
            'core boost speed (GHz)',
            'ram cap (GB)',
            'ssd (GB)', 'hdd (GB)', 
            'gpu size (MB)', 
            'gpu base speed (GHz)',
            'gpu boost speed (GHz)',
            'screen size (inch)',
            'screen horizontal resolution',
            'screen vertical resolution',
            'sRGB (%)',
            'weight (kg)',
            'battery capacity (WHr)']
unorder_cate_cols = ['producer', 'ram type', 'gpu prod', 'screen type', 'os']
order_cate_cols = ['intel_processor_model', 'amd_processor_model']

unorder_cate_cols_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))
column_transformer = make_column_transformer((SimpleImputer(strategy='mean'), nume_cols),
                                            (unorder_cate_cols_transformer, unorder_cate_cols),
                                            (SimpleImputer(strategy='most_frequent'), order_cate_cols))
preprocess_pipeline = make_pipeline(FunctionTransformer(split_processor_model), 
                                    ColAdderDropper(num_top_producers=4, num_top_os=4),
                                    column_transformer, StandardScaler())

preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)

In [24]:
preprocessed_train_X

array([[-0.38641678, -0.52661079,  0.7947706 , ..., -0.69388867,
         0.89990348, -0.38028954],
       [ 1.21649728,  1.03785628,  0.57681196, ...,  1.44115338,
         0.89990348, -0.38028954],
       [-0.38641678, -1.89551948, -1.16685713, ..., -0.69388867,
        -0.08822583, -0.38028954],
       ...,
       [ 0.41504025, -1.30884433, -2.69256758, ...,  1.44115338,
        -0.08822583, -0.38028954],
       [-0.38641678, -0.91772756, -0.73093985, ...,  1.44115338,
        -0.08822583, -0.38028954],
       [ 1.21649728,  0.45118113,  0.7947706 , ..., -0.69388867,
         0.89990348, -0.38028954]])

##### Trực quan hóa pipeline tiền xử lý

In [25]:
preprocess_pipeline

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function split_processor_model at 0x000001E46DB6BDC0>)),
                ('coladderdropper',
                 ColAdderDropper(num_top_os=4, num_top_producers=4)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(),
                                                  ['cores',
                                                   'core base speed (GHz)',
                                                   'core boost speed (GHz)',
                                                   'ram cap (GB)', 'ssd (GB)'...
                                                   'battery capacity (WHr)']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['producer', 'ram type',
                                                   'gpu prod', 'screen type',
                                                   'os']),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['intel_processor_model',
                                                   'amd_processor_model'])])),
                ('standardscaler', StandardScaler())])

---

# Mô hình hóa

## Sử dụng mô hình Linear Regression (hàm cực tiểu hóa là Stochastic Gradient Descent)

Sử dụng thuật toán Stochastic Gradient Descent cho bài toán Linear Regression

Chạy thử nghiệm các siêu tham số:
- `alpha`
- `num_top_producers`
- `num_top_os`

Chọn các giá trị của các siêu tham số cho độ lỗi nhỏ nhất trên tập validation

In [26]:
from sklearn.linear_model import SGDRegressor

In [27]:
# Hàm tính độ đo r^2
def compute_mse(y, preds):
    return ((y - preds) ** 2).mean()
def compute_rr(y, preds, baseline_preds):
    return 1 - compute_mse(y, preds) / compute_mse(y, baseline_preds)
baseline_preds = train_y_sr.mean()

In [28]:
full_pipeline = make_pipeline(FunctionTransformer(split_processor_model), 
                              ColAdderDropper(num_top_producers=4, num_top_os=4),
                              column_transformer, StandardScaler(),
                              SGDRegressor(penalty='l2', random_state=0))

train_errs = []
val_errs = []
alphas = [0.01, 0.1, 1, 10, 100]
num_top_producers_s = range(1, 8)
num_top_os_s = range(1, 7)
best_val_err = float('inf'); best_num_top_os = None; best_num_top_producers = None; best_alpha = None
for alpha in alphas:
    for num_top_producers in num_top_producers_s:
        for num_top_os in num_top_os_s:
            full_pipeline.set_params(coladderdropper__num_top_producers=num_top_producers, coladderdropper__num_top_os=num_top_os, sgdregressor__alpha=alpha)
            full_pipeline.fit(train_X_df, train_y_sr)
            train_errs.append(round(100 - compute_rr(train_y_sr, full_pipeline.predict(train_X_df), baseline_preds) * 100, 2))
            val_errs.append(round(100 - compute_rr(val_y_sr, full_pipeline.predict(val_X_df), baseline_preds) * 100, 2))
            if val_errs[-1] < best_val_err:
                best_val_err = val_errs[-1]
                best_alpha = alpha
                best_num_top_producers = num_top_producers
                best_num_top_os = num_top_os

In [29]:
print("val_errs:")
print(val_errs)

val_errs:
[30.83, 30.6, 30.62, 30.65, 30.61, 30.61, 30.83, 30.59, 30.6, 30.62, 30.59, 30.59, 31.04, 30.9, 30.95, 30.96, 30.93, 30.93, 30.82, 30.75, 30.81, 30.94, 30.9, 30.9, 30.73, 30.78, 30.96, 30.99, 30.95, 30.95, 30.73, 30.78, 30.96, 30.99, 30.95, 30.95, 30.73, 30.78, 30.96, 30.99, 30.95, 30.95, 30.34, 30.0, 30.09, 30.0, 29.99, 29.99, 30.35, 30.0, 30.08, 29.99, 29.99, 29.99, 30.56, 30.29, 30.42, 30.33, 30.33, 30.33, 29.79, 30.06, 30.2, 30.18, 30.18, 30.18, 30.19, 30.08, 30.19, 30.22, 30.21, 30.21, 30.19, 30.08, 30.19, 30.22, 30.21, 30.21, 30.19, 30.08, 30.19, 30.22, 30.21, 30.21, 32.36, 32.05, 31.97, 31.46, 31.46, 31.46, 32.38, 32.07, 31.99, 31.49, 31.49, 31.49, 32.66, 32.38, 32.31, 31.89, 31.88, 31.88, 32.3, 32.07, 32.03, 31.68, 31.68, 31.68, 31.89, 31.88, 31.81, 31.75, 31.74, 31.74, 31.89, 31.88, 31.81, 31.75, 31.74, 31.74, 31.89, 31.88, 31.81, 31.75, 31.74, 31.74, 60.79, 60.65, 60.55, 60.02, 60.0, 60.0, 60.72, 60.57, 60.47, 59.95, 59.93, 59.93, 60.85, 60.69, 60.59, 60.06, 60.04, 

In [30]:
full_pipeline = make_pipeline(FunctionTransformer(split_processor_model), 
                              ColAdderDropper(num_top_producers=4, num_top_os=4),
                              column_transformer, StandardScaler(),
                              SGDRegressor(penalty='l1', random_state=0))

train_errs = []
val_errs = []
alphas = [0.01, 0.1, 1, 10, 100]
num_top_producers_s = range(1, 8)
num_top_os_s = range(1, 7)
best_val_err = float('inf'); best_num_top_os = None; best_num_top_producers = None; best_alpha = None
for alpha in alphas:
    for num_top_producers in num_top_producers_s:
        for num_top_os in num_top_os_s:
            full_pipeline.set_params(coladderdropper__num_top_producers=num_top_producers, coladderdropper__num_top_os=num_top_os, sgdregressor__alpha=alpha)
            full_pipeline.fit(train_X_df, train_y_sr)
            train_errs.append(round(100 - compute_rr(train_y_sr, full_pipeline.predict(train_X_df), baseline_preds) * 100, 2))
            val_errs.append(round(100 - compute_rr(val_y_sr, full_pipeline.predict(val_X_df), baseline_preds) * 100, 2))
            if val_errs[-1] < best_val_err:
                best_val_err = val_errs[-1]
                best_alpha = alpha
                best_num_top_producers = num_top_producers
                best_num_top_os = num_top_os

In [31]:
print("val_errs:")
print(val_errs)

val_errs:
[30.99, 30.77, 30.78, 30.83, 30.79, 30.79, 30.99, 30.76, 30.76, 30.81, 30.77, 30.77, 31.2, 31.08, 31.12, 31.16, 31.12, 31.12, 31.0, 30.94, 30.99, 31.16, 31.12, 31.12, 30.91, 30.99, 31.19, 31.21, 31.17, 31.17, 30.91, 30.99, 31.19, 31.21, 31.17, 31.17, 30.91, 30.99, 31.19, 31.21, 31.17, 31.17, 30.95, 30.75, 30.77, 30.82, 30.78, 30.78, 30.98, 30.74, 30.75, 30.8, 30.76, 30.76, 31.19, 31.06, 31.11, 31.14, 31.1, 31.1, 30.98, 30.91, 30.89, 31.13, 31.09, 31.09, 30.88, 30.95, 31.15, 31.18, 31.14, 31.14, 30.88, 30.95, 31.15, 31.18, 31.14, 31.14, 30.88, 30.95, 31.15, 31.18, 31.14, 31.14, 30.82, 30.57, 30.64, 30.65, 30.6, 30.6, 30.84, 30.57, 30.62, 30.63, 30.59, 30.59, 31.03, 30.88, 30.99, 30.98, 30.95, 30.95, 30.84, 30.67, 30.79, 30.9, 30.85, 30.85, 30.74, 30.73, 30.87, 30.94, 30.9, 30.9, 30.74, 30.73, 30.87, 30.94, 30.9, 30.9, 30.74, 30.73, 30.87, 30.94, 30.9, 30.9, 30.94, 30.5, 30.7, 30.57, 30.52, 30.52, 30.96, 30.53, 30.7, 30.58, 30.55, 30.55, 31.05, 30.66, 30.49, 30.73, 30.31, 30.31

##### Lựa chọn mô hình

Rút ra nhận xét và chọn mô hình khi thử nghiệm với tham số `penalty` khác nhau (`l2` (mặc định của SGD) và `l1`): L2 xuất hiện các err rất cao (lên tới 100%), còn L1 thì các err không chênh lệch nhiều 
=> Cho thấy L2 xử lý vấn đề overfitting không tốt bằng L1

Vậy chọn`penalty` = L1 

In [32]:
print("Độ lỗi tốt nhất trên tập validation:")
print(best_val_err)
print("best_alpha:")
print(best_alpha)
print("best_num_top_producers:")
print(best_num_top_producers)
print("best_num_top_os:")
print(best_num_top_os)

Độ lỗi tốt nhất trên tập validation:
30.12
best_alpha:
10
best_num_top_producers:
4
best_num_top_os:
5


## Đánh giá mô hình tìm được

Huấn luyện lại `full_pipeline` trên `X_df` và `y_sr` (tập huấn luyện + tập validation) với `best_alpha`
, `best_num_top_producers` và `best_num_top_os` tìm được ở trên để ra được mô hình cụ thể cuối cùng

In [33]:
full_pipeline = make_pipeline(FunctionTransformer(split_processor_model), 
                              ColAdderDropper(num_top_producers=4, num_top_os=4),
                              column_transformer, StandardScaler(),
                              SGDRegressor(penalty='l1', random_state=0))
full_pipeline.set_params(coladderdropper__num_top_producers=num_top_producers, coladderdropper__num_top_os=num_top_os, sgdregressor__alpha=alpha)
full_pipeline.fit(X_df, y_sr)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function split_processor_model at 0x000001E46DB6BDC0>)),
                ('coladderdropper',
                 ColAdderDropper(num_top_os=6, num_top_producers=7)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(),
                                                  ['cores',
                                                   'core base speed (GHz)',
                                                   'core boost speed (GHz)',
                                                   'ram cap (GB)', 'ssd (GB)'...
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['producer', 'ram type',
                                                   'gpu prod', 'screen type',
                                                   'os']),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['intel_processor_model',
                                                   'amd_processor_model'])])),
                ('standardscaler', StandardScaler()),
                ('sgdregressor',
                 SGDRegressor(alpha=100, penalty='l1', random_state=0))])

#### Độ lỗi trên tập Test

In [34]:
# Đặt lại baseline_preds
baseline_preds = y_sr.mean()
100 - compute_rr(test_y_sr, full_pipeline.predict(test_X_df), baseline_preds) * 100

32.978945109747855

---

# NHÌN LẠI QUÁ TRÌNH LÀM ĐỒ ÁN

### Những khó khăn đã gặp phải:
- Trong quá trình tìm chủ đề: phải chọn được chủ đề phù hợp với khả năng (phải lấy được dữ liệu và tìm hiểu về các thuộc tính để xử lý dữ liệu)
- Trong quá trình thu thập dữ liệu: khó khăn trong việc tìm kiếm API (phải tìm được API đáng tin cậy, dữ liệu rõ ràng) và lấy dữ liệu tự động từ API - số lượt request tối đa mỗi ngày chỉ có 1000 lần (1000 lần thì chỉ lấy được khoảng 400-500 mẫu) dẫn đến tốn thời gian trong việc thu thập dữ liệu
- Trong quá trình tiền xử lý: không có khó khăn
- Trong quá trình mô hình hóa: Khó khăn trong việc lựa chọn mô hình và các tham số cho mô hình cần phải tìm hiểu để chọn cho hợp lý: Mô hình MLP không hội tụ được và cho độ lỗi quá cao, Mô hình SGD với tham số penalty mặc định (L2) thì gặp phải vấn đề overfitting

### Những gì đã học được:
- Hiểu rõ thêm về quy trình thu thập, xử lý và mô hình hóa dữ liệu
- Học được cách làm việc nhóm trên Github
- Cách code để thu thập dữ liệu tự động từ API
- Hiểu thêm về các thuật toán mô hình hóa và các tham số
- Không phải model có độ lỗi trên tập validation thấp thì sẽ tốt khi chạy trên tập kiểm tra

### Những cải thiện nếu có thêm thời gian:
- Cải thiện mô hình:
    - Thu thập thêm dữ liệu
    - kiểm tra lại dữ liệu đã thu thập, bỏ những mẫu dữ liệu nhiễu, không chính xác
    - Tìm hiểu thêm để lựa chọn thêm thuộc tính hoặc xóa bỏ các thuộc tính không cần thiết
    - Tìm hiểu thêm về các thuật toán mô hình hóa để lựa chọn và đặt các tham số phù hợp với bài toán hơn
- Viết slide báo cáo hoàn chỉnh hơn
- Tìm hiểu thêm về Github để làm việc nhóm hiệu quả hơn

---

# TÀI LIỆU THAM KHẢO

1. scikit-learn.org
2. Slide bài giảng môn "Nhập môn Khoa học Dữ liệu" của thầy Nguyễn Trần Trung Kiên
3. Các bài tập và các file Demo môn "Nhập môn Khoa học Dữ liệu" của thầy Nguyễn Trần Trung Kiên